In [1]:
import os

import torch
import torch.optim as optim
from torch.utils.data import random_split

from dataset import MiniFlickrDataset, get_loader
from todo import CaptioningModel
from trainer import Trainer
from lr_warmup import LRWarmup
import utils

/Users/aashrith/VirtualEnvs/env-dev/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/aashrith/VirtualEnvs/env-dev/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class Args:
    def __init__(self):
        self.data_path = 'dataset.pkl'
        self.clip_model = 'openai/clip-vit-base-patch32'
        self.text_model = 'gpt2'
        self.seed = 100
        self.num_workers = 0
        self.train_size = 0.84
        self.val_size = 0.13
        self.test_size = 100
        self.epochs = 15
        self.lr = 3e-3
        self.k = 0.33
        self.batch_size_exp = 6
        self.ep_len = 4
        self.num_layers = 6
        self.n_heads = 16
        self.forward_expansion = 4
        self.max_len = 40
        self.dropout = 0.1
config = Args()

In [3]:
device = utils.init_env(config.seed)

# Create data loaders
dataset = MiniFlickrDataset(config.data_path)
config.train_size = int(config.train_size * len(dataset))
config.val_size = len(dataset) - config.train_size - config.test_size
train_dataset, val_dataoet, test_dataset = random_split(dataset, [config.train_size, config.val_size, config.test_size])
train_loader = get_loader(
    train_dataset, 
    bs_exp=config.batch_size_exp, 
    shuffle=True, 
    num_workers=config.num_workers,
    pin_memory=True,
)
test_loader = get_loader(
    test_dataset,
    bs_exp=0,
    shuffle=False,
    num_workers=config.num_workers,
    pin_memory=True,
    train=False,
)

# Creat model
model = CaptioningModel(
    clip_model=config.clip_model,
    text_model=config.text_model,
    ep_len=config.ep_len,
    num_layers=config.num_layers, 
    n_heads=config.n_heads, 
    forward_expansion=config.forward_expansion, 
    dropout=config.dropout, 
    max_len=config.max_len,
    device=device
)

# Create optimizer, lr scheduler
optimizer = optim.Adam(model.parameters(), lr=config.lr)
warmup = LRWarmup(epochs=config.epochs, max_lr=config.lr, k=config.k)
scheduler = optim.lr_scheduler.LambdaLR(optimizer, warmup.lr_warmup)

# Create trainer
trainer = Trainer(
    model=model,
    optimizer=optimizer,
    scaler=torch.cuda.amp.GradScaler(),
    scheduler=scheduler,
    train_loader=train_loader,
    test_loader=test_loader,
    device=device
)

# use _load_ckpt method of the trainer to load weights from the saved checkpoint to resume the training. Below is a sample code for the same

#trainer._load_ckp("path to .pt file")

/var/folders/z8/_9084bwd59j3c9ln84x1klyw0000gn/T/ipykernel_7661/201511195.py:46: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler=torch.cuda.amp.GradScaler(),
/Users/aashrith/VirtualEnvs/env-dev/lib/python3.9/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


In [4]:
# i = 0
# for img_emb, input_ids, attention_mask in train_loader:

#     print(img_emb.shape)
#     print(input_ids.shape)
#     print(attention_mask.shape)

#     loss = model.train_forward(
#         img_emb= img_emb.to(device),
#         trg_cap= input_ids.to(device),
#         att_mask= attention_mask.to(device)
#     )

#     print('\n', loss)

#     if i > 1:
#         break

#     i += 1
#     print()


# # model.train_forward(
# #     img_emb=
# # )

In [6]:
# Start training
for epoch in range(trainer.epoch, config.epochs):
    trainer.train_epoch()

    score = trainer.test_epoch()
    print("Score: {:.4f}".format(score))

    os.makedirs("checkpoints", exist_ok=True)
    if (epoch + 1) % 3 == 0:
        trainer.save_ckp(os.path.join("checkpoints", f'epoch_{epoch + 1}.pt'))

Epoch: 1 | Loss: ---:   0%|          | 0/2086 [00:00<?, ?it/s]/Users/aashrith/Desktop/Code/VisionAndLanguage/HW2_Captioning/trainer.py:54: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
/Users/aashrith/VirtualEnvs/env-dev/lib/python3.9/site-packages/torch/amp/autocast_mode.py:265: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Epoch: 1 | Loss: 10.803:   0%|          | 1/2086 [00:23<13:44:48, 23.74s/it]


KeyboardInterrupt: 